In [1]:
import sys
import os
import random as r

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
%matplotlib inline


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.activations import relu

# Add the architecture path for the DenseEncoderDecoder and NMSE
sys.path.append("../architecture/")
from Conv3DEncoderDecoder import Conv3DEncoder, Conv3DDecoder
from NormalizedMeanSquaredError import NormalizedMeanSquaredError2D as NMSE
sys.path.append("../tools/")
from utils import run_experiment
from cubes2npy import CubeFiles


2023-05-21 21:57:33.305216: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-21 21:57:33.338684: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 21:57:35.665418: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-21 21:57:35.684970: I tens

In [2]:
cubes=CubeFiles()
files=sorted(os.listdir(os.path.join('..','data','cis-DCE-hann-x-10k')))
for f in files :
    cubes.read_cube(fname=os.path.join('..','data','cis-DCE-hann-x-10k',f))

cube_data=cubes.density_as_numpy()
print(cube_data.shape)

Reading :  ../data/cis-DCE-hann-x-10k/density.0000000001.cube  as cube number 1
Reading :  ../data/cis-DCE-hann-x-10k/density.0000000100.cube  as cube number 2
Reading :  ../data/cis-DCE-hann-x-10k/density.0000000200.cube  as cube number 3
Reading :  ../data/cis-DCE-hann-x-10k/density.0000000300.cube  as cube number 4
Reading :  ../data/cis-DCE-hann-x-10k/density.0000000400.cube  as cube number 5
Reading :  ../data/cis-DCE-hann-x-10k/density.0000000500.cube  as cube number 6
Reading :  ../data/cis-DCE-hann-x-10k/density.0000000600.cube  as cube number 7
Reading :  ../data/cis-DCE-hann-x-10k/density.0000000700.cube  as cube number 8
Reading :  ../data/cis-DCE-hann-x-10k/density.0000000800.cube  as cube number 9
Reading :  ../data/cis-DCE-hann-x-10k/density.0000000900.cube  as cube number 10
Reading :  ../data/cis-DCE-hann-x-10k/density.0000001000.cube  as cube number 11
Reading :  ../data/cis-DCE-hann-x-10k/density.0000001100.cube  as cube number 12
Reading :  ../data/cis-DCE-hann-x-10k

In [3]:
#trim the size
cubes.trim_cube_data(30,70,30,70,30,70)
cube_data=np.moveaxis(cubes.density_as_numpy(),-1,0)
cube_data=tf.expand_dims(cube_data,axis=4)
print(cube_data.shape)


(500, 40, 40, 40, 1)


In [60]:
class conv_autoencoder(tf.keras.Model):
    def __init__(self, input_shape,num_filters,latent_space_size):
        super(conv_autoencoder, self).__init__()
        self.latent_dim = latent_space_size
        self.data_shape = input_shape
        self.activation = activation
        self.num_filters=num_filters
        self.encoded=None
        self.decoded=None
        self.latent_shape=None
        print(self.data_shape[1:])

        self.encoder_layers = [tf.keras.layers.Conv3D(filters=self.num_filters[0],kernel_size=4,strides=1,padding= 'SAME',activation=relu,kernel_initializer=keras.initializers.VarianceScaling(),kernel_regularizer=l1_l2(0, 1e-6),input_shape=self.data_shape[1:])]
        for filters in num_filters[1:]:
            self.encoder_layers.append(tf.keras.layers.AveragePooling3D(pool_size=2,strides=2,padding='VALID'))
            self.encoder_layers.append(tf.keras.layers.Conv3D(filters=self.num_filters[0],kernel_size=4,strides=1,padding= 'SAME',activation=relu,kernel_initializer=keras.initializers.VarianceScaling(),kernel_regularizer=l1_l2(0, 1e-6)))

        self.decoder_layers=[tf.keras.layers.Conv3D(filters=self.num_filters[-1],kernel_size=4,strides=1,padding= 'SAME',activation=relu,kernel_initializer=keras.initializers.VarianceScaling(),kernel_regularizer=l1_l2(0, 1e-6))]
        for filters in num_filters[-2::-1]:
            self.decoder_layers.append(tf.keras.layers.Conv3DTranspose(filters=filters,kernel_size=4,strides=2,padding= 'SAME',activation=relu,kernel_initializer=keras.initializers.VarianceScaling(),kernel_regularizer=l1_l2(0, 1e-6)))
        self.decoder_layers.append(tf.keras.layers.Conv3DTranspose(filters=1,kernel_size=4,strides=1,padding= 'SAME',activation=relu,kernel_initializer=keras.initializers.VarianceScaling(),kernel_regularizer=l1_l2(0, 1e-6)))




    def call(self, x):
        for layer in self.encoder_layers:
            x = layer(x)
        self.encoded = x
        for layer in self.decoder_layers:
            x=layer(x)
        self.decoded=x

        return self.decoded

In [61]:
input_shape=cube_data.shape
print(input_shape)
filters=[8, 16, 32, 64]
A = conv_autoencoder(input_shape=input_shape, num_filters=filters, latent_space_size=10)
A.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

(500, 40, 40, 40, 1)
(40, 40, 40, 1)


In [62]:
A.fit(cube_data, cube_data, epochs=50, shuffle=True)

Epoch 1/50
16/16 [==============================] - 3s 42ms/step - loss: 0.3501
Epoch 2/50
16/16 [==============================] - 1s 42ms/step - loss: 0.2071
Epoch 3/50
16/16 [==============================] - 1s 42ms/step - loss: 0.1643
Epoch 4/50
16/16 [==============================] - 1s 41ms/step - loss: 0.1636
Epoch 5/50
16/16 [==============================] - 1s 41ms/step - loss: 0.1635
Epoch 6/50
16/16 [==============================] - 1s 42ms/step - loss: 0.1635
Epoch 7/50
16/16 [==============================] - 1s 42ms/step - loss: 0.1635
Epoch 8/50
16/16 [==============================] - 1s 41ms/step - loss: 0.1635
Epoch 9/50
16/16 [==============================] - 1s 42ms/step - loss: 0.1635
Epoch 10/50
16/16 [==============================] - 1s 40ms/step - loss: 0.1635
Epoch 11/50
16/16 [==============================] - 1s 41ms/step - loss: 0.1635
Epoch 12/50
16/16 [==============================] - 1s 41ms/step - loss: 0.1635
Epoch 13/50
16/16 [==================

In [63]:
g = A.predict(cube_data)
error = np.abs(cube_data - g)

16/16 [==============================] - 0s 23ms/step
